# CS 436 CS5310 - Computer Vision - Assignment#4 - Part#1

*__Submission Instructions:__*
- Rename this notebook to `PA4_rollnumber.ipynb` before submission on LMS.
- Code for all the tasks must be written in this notebook (you do not need to submit any other files).
- The output of all cells must be present in the version of the notebook you submit.
- The university honor code should be maintained. Any violation, if found, will result in disciplinary action.

In [1]:
#Executed on Colab
#Import all the required libraries
import keras
from keras.datasets import fashion_mnist
from keras.layers import Activation, Input, Embedding, LSTM, Dense, Lambda, GaussianNoise, concatenate
from keras.models import Model
import numpy as np
from keras.utils import np_utils
from keras.layers.core import Dense, Dropout, Activation
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adam, RMSprop
from keras.constraints import max_norm
from keras.layers import MaxPooling2D, Dropout, Dense, Flatten, Activation, Conv2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.losses import categorical_crossentropy as logloss
from keras.metrics import categorical_accuracy
from keras.applications import vgg16
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from matplotlib import offsetbox
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sn
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Overview

In this assignment you will be exploring a few important concepts used in the deep learning projects:
- Training image classification algorithms using Deep Learning
- Dataset Analyses 
- Testing deep learning classifier with the test data
- Fine-tuning / Transfer Learning

We will be using the Fashion MNIST dataset that is provided by Keras. You will also be working with pretrained models, which can be downloaded from keras applications. You are **highly** encouraged to explore the images in dataset and model architectures in order to get the most out of this assignment. 

**_Dataset:_**
Fashion-MNIST is a dataset of  article images from a e-commerce website named zalando with 70,000 images of size (28x28x1) each belonging to one of the 10 classes.

**_Pretrained Models:_** 
Can be found [here](https://keras.io/applications/#applications)

## Task 1: Data Preparation for transfer learning

To prepare your data:
- Download the Fashion MNIST dataset
- Split it into train, validation, and test sets
- Convert the labels into categorical one hot encoded labels
- Cast the images as float and normalize the values between zero and one
- The Fashion MNIST dataset contains images of shape (28,28,1), the mimumm (height,width,channels) required by our pretrained model(in this case vgg-16) is (32,32,3)
- Convert the images in the datset to shape (32,32,3)

In [2]:
nb_classes = 10
#Download Fashion MNIST dataset(Done for you)
(X_Train, y_Train), (X_test, y_test) = fashion_mnist.load_data()
#Split the fashion MNIST dataset into train, validation and test sets (Done for you)
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)

#Convert y_train,y_val and y_test to categorical binary values 
#TO DO
y_train = keras.utils.to_categorical(y_train, nb_classes, dtype='float32')
y_val = keras.utils.to_categorical(y_val, nb_classes, dtype='float32')
y_test = keras.utils.to_categorical(y_test, nb_classes, dtype='float32')
#Hint
#See function "np_utils.to_categorical()"

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
#Reshape your dataset to batch_size, width, height, #channels(Done for you)
X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)

#Convert your dataset to float type
#TO DO
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')
#Normalize the values
X_train = cv2.normalize(X_train, None, 0, 255, cv2.NORM_MINMAX)
X_val = cv2.normalize(X_val, None, 0, 255, cv2.NORM_MINMAX)
X_test = cv2.normalize(X_test, None, 0, 255, cv2.NORM_MINMAX)

#TO DO

In [0]:
#Preprocess the data for our pretrained model(requires 3 channels) and a mimimum height and width of 32.
#Convert all images to shape (32,32,3)
X_train_new=[]
X_val_new=[]

X_train_new

size = len(X_train)

for i in range(size):
  X_train_new.append(cv2.resize(X_train[i], (32, 32)))
  val = np.empty((32, 32, 3), dtype = 'float32')
  for j in range(32):
    for k in range(32):
      val[j][k][0] = X_train_new[i][j][k] 
      val[j][k][1] = X_train_new[i][j][k]
      val[j][k][2] = X_train_new[i][j][k]
  X_train_new[i] = val

size = len(X_val)
for i in range(size):
  X_val_new.append(cv2.resize(X_val[i], (32, 32)))
  val = np.empty((32, 32, 3), dtype = 'float32')
  for j in range(32):
    for k in range(32):
      val[j][k][0] = X_val_new[i][j][k] 
      val[j][k][1] = X_val_new[i][j][k]
      val[j][k][2] = X_val_new[i][j][k]
  X_val_new[i] = val


#TO DO

In [0]:
def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet
    Args:
    base_model: keras model excluding top layer
    nb_classes: number of classes
    Returns:
    new keras model with last fully connected layer
    """
    FC_SIZE=1024
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) 
    predictions = Dense(nb_classes, activation='softmax')(x) 
    model = Model(input=base_model.input, output=predictions)
    return model

## Task 2: Transfer Learning

Next you will employ Transfer Learning and finetune the pretrained vgg-16 model to better fit the Fashion MNIST data for 10 classes. You will:

- Change the number of nodes in the last FC layer according to the number of classes i.e. 10 
- Freeze everything except the FC layers and train it using the train split of our dataset (using appropriate hyperparameters), validating the network for validation split of data.
- Plot loss/accuracy vs epochs curves for your simulation



In [6]:
#Load the keras base model excluding the top layer
#TO DO
#Hint
#base_model=vgg16.VGG16(...)

base_model = vgg16.VGG16(include_top = False, weights = 'imagenet', input_tensor = None, input_shape = None, pooling = None, classes = nb_classes)

58892288/58889256 [==============================] - 2s 0us/step


In [7]:
#Since we will be using the pretrained weights for the image net dataset we will freeze all layers of our base model(Done for you)
for layer in base_model.layers:
    layer.trainable = False
#We will now add a new last layer according to the number of classes in our dataset using the "add_new_last_layer()" function provided to you
#TO DO
#Hint
#vgg_imagenet_new=add_new_last_layer(base_model,...)
vgg_imagenet_new = add_new_last_layer(base_model, nb_classes)

#Print Model Summary(Done for you)
vgg_imagenet_new.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [0]:
#Intialize Optimizer for our model(Done for you). You can try using different optimizers with different values
adam=Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#Compile Model
#TO DO
#Hint
#vgg_imagenet_new.compile(optimizer=adam,...)
vgg_imagenet_new.compile(optimizer = adam, loss = logloss, metrics = ['accuracy'], weighted_metrics = None, target_tensors = None)

In [9]:
#Train the vgg-16 model
epochs = 5
batch_size = 32 
#TO DO
#Hint
#history=vgg_imagenet_new.fit(...)
history = vgg_imagenet_new.fit(x = np.array(X_train_new), y = y_train, batch_size = batch_size, epochs = epochs,  validation_data = (np.array(X_val_new), y_val))
#The history variable will be used later to plot accuracy and loss curves

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 29s 602us/step - loss: 0.9227 - accuracy: 0.7976 - val_loss: 0.4757 - val_accuracy: 0.8307
Epoch 2/5
48000/48000 [==============================] - 22s 464us/step - loss: 0.4330 - accuracy: 0.8461 - val_loss: 0.4724 - val_accuracy: 0.8362
Epoch 3/5
48000/48000 [==============================] - 22s 466us/step - loss: 0.3970 - accuracy: 0.8588 - val_loss: 0.4846 - val_accuracy: 0.8361
Epoch 4/5
48000/48000 [==============================] - 22s 465us/step - loss: 0.3771 - accuracy: 0.8644 - val_loss: 0.4852 - val_accuracy: 0.8425
Epoch 5/5
48000/48000 [==============================] - 22s 464us/step - loss: 0.3495 - accuracy: 0.8745 - val_loss: 0.4893 - val_accuracy: 0.8403


In [0]:
#Lets test our transfer learned model on the test set
#Preprocess the test data just like we did before i.e convert all images to shape (32,32,3)
X_test_new = []
size = len(X_test)
for i in range(size):
  X_test_new.append(cv2.resize(X_test[i], (32, 32)))
  val = np.empty((32, 32, 3), dtype ='float32')
  for j in range(32):
    for k in range(32):
      val[j][k][0] = X_test_new[i][j][k] 
      val[j][k][1] = X_test_new[i][j][k]
      val[j][k][2] = X_test_new[i][j][k]
  X_test_new[i] = val

In [0]:
#Evaluate the transfer learned model
#TO DO
#Hint
#vgg_imagenet_new.evaluate(...)
vgg_imagenet_new.evaluate(x = np.array(X_test_new), y = y_test, batch_size = 32, sample_weight=None, steps=None, callbacks=None)

In [0]:
#Plot training and validation accuracy curves using matplot lib
#Hint
#Values for training and validation accuarcy can be obtained using history.history['accuracy'] and history.history['val_accuracy'] respectively
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

plt.plot(training_accuracy)
plt.plot(validation_accuracy)
plt.show

In [0]:
#Plot training and validation loss curves using matplot lib
#Hint
#Values for training and validation loss can be obtained using history.history['loss'] and history.history['val_loss'] respectively
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

plt.plot(training_loss)
plt.plot(validation_loss)
plt.show

## Task 3: Network Evaluation

Next you will test your finetuned model by plotting a confusion matrix between classes predicted. You will:

- Test your model for images in the test set
- Construct a multiclass confusion matrix (for 10 classes) for actual and predicted class of each image and visualize the confusion matrix as a heatmap

*You can use scikit-learn's `metrics.confusion_matrix` function. Consult the relevant documentation.* 

In [0]:
y_true = y_test.argmax(axis = 1)
y_pred = vgg_imagenet_new.predict(np.array(X_test_new), verbose = 1).argmax(axis = 1)
conf_matrix = confusion_matrix(y_true, y_pred)

sn.heatmap(conf_matrix, xticklabels = [0, 1, 2, 3, 4, 5 ,6 ,7, 8, 9], yticklabels = [0, 1, 2, 3, 4, 5 ,6 ,7, 8, 9])
plt.show